# Lab 1 - drzewa decyzyjne
*"pro" version*

Autorstwa:
- Wojciech Kot  151879
- Julia Samp    151775
z grupy L7, zajęcia piątkowe 11:45

Step - by - step - Wersja podstawowa (pro znajduje się poniżej)

Wpierw, wgranie danych do dataframe'u zapewnianego przez pandas
(ze względu na wygodę, jaką zapewnia oraz obsługę .csv)

In [3]:
import pandas


def prepare_pandas_df_for_basic_problem(path):
    data = pandas.read_csv(path)
    data = data.drop(['PassengerId', 'Name'], axis=1)
    data['Age'] = pandas.cut(data['Age'], bins=[0, 20, 40, 100], labels=['young', 'middle', 'old'], right=True)
    return data


dane2 = prepare_pandas_df_for_basic_problem("titanic-homework.csv")
print(dane2)

    Pclass     Sex     Age  SibSp  Parch  Survived
0        3    male  middle      1      0         0
1        1  female  middle      1      0         1
2        3  female  middle      0      0         1
3        1  female  middle      1      0         1
4        3    male  middle      0      0         0
..     ...     ...     ...    ...    ...       ...
95       3    male     old      0      0         0
96       1    male     old      0      0         0
97       1    male  middle      0      1         1
98       2  female  middle      0      1         1
99       2    male  middle      1      0         0

[100 rows x 6 columns]


Kiedy mamy już poprawnie załadowane dane, należy zacząć budować drzewo, jednak najpierw kilka funkcji, które mogą nam się przydać:

possible_values() oraz get_column_names() służą jedynie temu, aby dalszy kod był bardziej czytelny i obsługują dataframe.
possible_values zwraca listę wszystkich unikalnych wartości, jakie występują w danej kolumnie (a więc jakie są dostępne dla tego parametru)
Get_column_names natomiast, zgodnie ze swoją nazwą, zwraca nazwy kolumn dataframe'u, czyli nasze nazwy parametrów


# TODO
entropy etc - #todo opisać dalej

In [ ]:
import math


def possible_values(df, column_name):
    return df[column_name].unique().tolist()


def get_column_names(df):
    return df.columns.tolist()


def entropy(lista):
    e = 0
    suma = sum(lista)
    for i in lista:
        if i != 0:
            e -= (i / suma) * math.log(i / suma, 2)
    return e


def conditional_entropy(dane):
    ce = 0
    ile = 0
    for i in dane:
        ile += sum(i)
    for i in dane:
        e = entropy(i)
        ce += e * sum(i) / ile
    return ce


def information_gain(dane):
    y = 0
    n = 0
    for i in dane:
        y += i[0]
        n += i[1]
    ig = entropy([y, n]) - conditional_entropy(dane)

    return ig


def gain_ratio(dane):
    ig = information_gain(dane)
    ii = 0
    ile = 0
    for i in dane:
        ile += sum(i)
    for i in dane:
        ii -= (sum(i) / ile) * math.log(sum(i) / ile, 2)
    gr = ig/ii
    return gr


def filter_count(df, column_name, value):
    filtered_df = df[df[column_name] == value]
    p = int(filtered_df['Survived'].sum())
    ile_wszystkich = len(filtered_df)
    # zwraca parę [przeżyło, nie-przeżyło]
    return [p, ile_wszystkich - p]


def split(df, column_name):
    l = []
    for val in possible_values(df, column_name):
        l.append(filter_count(df, column_name, val))
    return l

A tutaj już główny kod programu, czyli po kolei kroki jakie wykonujemy aby zbudować nasze drzewo decyzyjne

In [ ]:
# todo - main i takie tam